In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import time
from sklearn import metrics
from scipy.optimize import brute, minimize
from sklearn.metrics import make_scorer
from joblib import dump, load
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def get_score(outputs, targets):
    mcrmse = []
    for i in range(2):
        mcrmse.append(
            metrics.mean_squared_error(
                targets[:, i],
                outputs[:, i],
                squared=False,
            ),
        )
    mcrmse = np.mean(mcrmse)
    return mcrmse


def obj_func(x, *oof_list):
    
    
    
    oof = 0
    x[x<0] = 0
    x = x/np.sum(x)
    for i in range(min(len(x), len(oof_list))):
        oof += x[i]*oof_list[i][['pred_0', 'pred_1']]
    ground_truth = oof_list[0][['content', 'wording']]
    return get_score(oof.values, ground_truth.values)

In [3]:
pred_cols = ['pred_0', 'pred_1']
target_cols = ['content', 'wording']

In [4]:
Data_Path = "C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/CommonLit - Evaluate Student Summaries/input/commonlit-evaluate-student-summaries/"
oof_024 = pd.read_csv('../024/Comonlist_024oof_df.csv', usecols=['student_id', 'fold']+pred_cols)

oof_024 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024, on='student_id')
print(oof_024.shape)

oof_024_2 = pd.read_csv('../024_2/Comonlist_024_2_oof_df.csv', usecols=['student_id', 'fold']+pred_cols)
oof_024_2 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024_2, on='student_id')
print(oof_024_2.shape)

oof_024_3 = pd.read_csv('../024_3/Comonlist_024_3_oof_df.csv', usecols=['student_id', 'fold']+pred_cols)
oof_024_3 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024_3, on='student_id')
print(oof_024_3.shape)


oof_tuple = tuple([oof_024, oof_024_2, oof_024_3])

for i in range(len(oof_tuple)):
    print(get_score(oof_tuple[i][['pred_0', 'pred_1']].values, oof_tuple[0][['content', 'wording']].values))
    

st = time.time()
res = minimize(obj_func, x0=np.ones((len(oof_tuple), )), args=oof_tuple, method='Nelder-Mead')
print((time.time()-st), 'sec.')

print('The optimal weights are', res.x)
print('The optimal OOF MCRMSE is', res.fun)


(7165, 8)
(7165, 8)
(7165, 8)
0.5597027507453362
0.5605429571763885
0.5578805947858928
0.2074449062347412 sec.
The optimal weights are [0.81914641 1.32156335 0.85428778]
The optimal OOF MCRMSE is 0.5520873341760086


# nelder-mead

In [5]:
st = time.time()
res = minimize(obj_func, x0=np.ones((len(oof_tuple), )), args=oof_tuple, method='Nelder-Mead')
print((time.time()-st), 'sec.')

print('The optimal weights are', res.x)
print('The optimal OOF MCRMSE is', res.fun)


0.2074742317199707 sec.
The optimal weights are [0.81914641 1.32156335 0.85428778]
The optimal OOF MCRMSE is 0.5520873341760086


In [6]:
w = res.x
sum_w = sum(w)
norm_w = [w/sum_w for w in w]
print(sum(norm_w))
w = [w/sum_w for w in w]
w

1.0


[0.27350486989087774, 0.4412569057496202, 0.2852382243595021]

# BaysianRidge

In [35]:
# oof_avg_preds = 0.45 * (0.6 * oof_024[pred_cols].values + 0.1 * oof_026[pred_cols].values + 0.05 * oof_n004[pred_cols].values + 0.05 * oof_n005[pred_cols].values + 0.05 * oof_kojimar[pred_cols].values + 0.05 * oof_n009[pred_cols].values + 0.05 * public_list[0][pred_cols].values + 0.05 * public_list[1][pred_cols].values) \
#           + 0.4 * oof_tf[pred_cols].values + 0.15 * (0.8 * oof_svr[pred_cols].values +  + 0.2 * oof_xgb[pred_cols].values)

# oof_avg_preds = 0.45 * (0.6 * oof_024[pred_cols].values + 0.1 * oof_026[pred_cols].values + 0.05 * oof_n004[pred_cols].values + 0.05 * oof_n005[pred_cols].values + 0.05 * oof_kojimar[pred_cols].values + 0.05 * oof_n009[pred_cols].values + 0.05 * public_list[0][pred_cols].values + 0.05 * public_list[1][pred_cols].values) \
#           + 0.4 * oof_tf[pred_cols].values  + 0.2 * oof_xgb[pred_cols].values

# get_score(oof_avg_preds, oof_024[target_cols].values)

0.4714819803882077

In [28]:
# oof_opt_preds = 0
# for wgt, oof_df in zip(w, oof_tuple):
#     oof_opt_preds += wgt * oof_df[pred_cols].values
    
# get_score(oof_opt_preds, oof_024[target_cols].values)

0.44352029296871787

In [7]:
preds_024 = pd.read_csv(f'{Data_Path}sample_submission.csv')[target_cols].values
preds_024_2 = pd.read_csv(f'{Data_Path}sample_submission.csv')[target_cols].values
preds_024_3 = pd.read_csv(f'{Data_Path}sample_submission.csv')[target_cols].values


In [36]:
# avg_preds = 0.45 * (0.6 * preds_024 + 0.1 * preds_026 + 0.05 * preds_n004 + 0.05 * preds_n005 + 0.05 * kojimar_preds + 0.05 * preds_n009 + 0.05 * preds_1 + 0.05 * preds_2) + 0.4 * tf_preds + 0.15 * (0.8 * svr_preds + 0.2 * xgb_preds)
# avg_preds = 0.45 * (0.6 * preds_024 + 0.1 * preds_026 + 0.05 * preds_n004 + 0.05 * preds_n005 + 0.05 * kojimar_preds + 0.05 * preds_n009 + 0.05 * preds_1 + 0.05 * preds_2) + 0.4 * tf_preds + 0.15 * + 0.2 * xgb_preds
# avg_preds

array([[2.64, 2.64, 2.64, 2.64, 2.64, 2.64],
       [2.64, 2.64, 2.64, 2.64, 2.64, 2.64],
       [2.64, 2.64, 2.64, 2.64, 2.64, 2.64]])

In [9]:
oof_tuple[0]

,student_id,prompt_id,text,content,wording,fold,pred_0,pred_1
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,3,-0.120802,1.083543
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,2,-0.786129,-0.261221
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,1,2.283512,2.216042
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,1,-0.880990,-0.751035
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,3,2.052107,2.196688
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,2,-0.309670,-0.187040
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,1,-0.419553,-0.221060
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,1,-0.964639,-0.558654
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,0,-0.148224,0.386016


In [27]:
from joblib import dump, load
import sys
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.linear_model import BayesianRidge

TRAIN = []
for oof in oof_tuple:
    # oof = pd.merge(pd.read_csv('../input/feedback-prize-english-language-learning/train.csv'), oof, on=['text_id']+target_cols)
    TRAIN.append(oof[pred_cols].values)

oof = pd.read_csv('../024/Comonlist_024oof_df.csv')
# oof = pd.merge(pd.read_csv('../input/feedback-prize-english-language-learning/train.csv'), oof, on='text_id')

TEST = np.concatenate([
    preds_024, preds_024_2, preds_024_3
], 1)


# N_SPLITS = 10

# skf = MultilabelStratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
# for i,(train_index, val_index) in enumerate(skf.split(oof,oof[target_cols])):
#     oof.loc[val_index,'fold'] = i

N_SPLITS = 4
TRAIN = np.concatenate(TRAIN, 1)

ypredtrain_ = np.zeros((TRAIN.shape[0], len(target_cols)))
ypredtarget_ = np.zeros((TRAIN.shape[0], len(target_cols)))
ypredtest_ = np.zeros((TEST.shape[0], len(target_cols)))

for fold in range(N_SPLITS):
    ind_train = oof['fold'] != fold
    ind_valid = oof['fold'] == fold

    model = BayesianRidge()
    model = MultiOutputRegressor(model)
    model.fit(TRAIN[ind_train], oof.loc[ind_train, target_cols].values)
    dump(model, f'BayesianRidge_{fold}.model')
    
    ypredtrain_[ind_valid, :] = model.predict(TRAIN[ind_valid])
    ypredtest_ += model.predict(TEST)
    ypredtarget_[ind_valid, :] = oof.loc[ind_valid, target_cols].values
    del model; gc.collect()
    
print(get_score(ypredtrain_, ypredtarget_))
print()

1.049916400537004

